In [1]:
import cv2
from shapely.strtree import STRtree
import matplotlib.pyplot as plt
import csv
import numpy as np
import geopandas
import os
from shapely.ops import unary_union
from shapely.geometry import Polygon, MultiPolygon
import glob
import json
import openslide
from tqdm import tqdm

In [15]:
#function
def get_structure(load_dict,classname):
    structure = []
    if len(classname) == 1:
        for load_dict_i in tqdm(load_dict, desc=f"Extracting {classname}"):
            if 'classification' in  load_dict_i['properties']:
                prop_name = load_dict_i['properties']['classification']['name']
                if (prop_name == classname[0]):
                    contours = load_dict_i['geometry']['coordinates']
                    if (load_dict_i['geometry']['type'] == 'MultiPolygon'):
                        for contour in contours:
                            points = [pt for pt in contour[0]]
                            structure.append(Polygon(points))
                    elif (load_dict_i['geometry']['type'] == 'Polygon'):
                        for contour in contours:
                            points = [pt for pt in contour]
                            structure.append(Polygon(points))
    elif(len(classname)==2):
        for load_dict_i in tqdm(load_dict, desc=f"Extracting {classname}"):
            if 'classification' in  load_dict_i['properties']:
                prop_name = load_dict_i['properties']['classification']['name']
                if (prop_name == classname[0] or prop_name==classname[1]):
                    contours = load_dict_i['geometry']['coordinates']
                    if (load_dict_i['geometry']['type'] == 'MultiPolygon'):
                        for contour in contours:
                            points = [pt for pt in contour[0]]
                            structure.append(Polygon(points))
                    elif (load_dict_i['geometry']['type'] == 'Polygon'):
                        for contour in contours:
                            points = [pt for pt in contour]
                            structure.append(Polygon(points))
    # result = structure[0]
    # for polygon in tqdm(structure[1:],desc=f'Post processing the {classname},N=={len(structure)}'):
    #     result = result.union(polygon)
    return structure


In [24]:
json_files = glob.glob("/Users/fanfan/Desktop/tubule_paper_KI/KI_resubmission/bar_figure/json/*.json")
for json_file in json_files:
    wsiid = os.path.basename(json_file).split('.json')[0]
    wsi_file = glob.glob(f"/Volumes/data/Neptune/FSGS_MCD/USE_TUBULE/{wsiid}*")[0]
    with open(json_file, 'r') as load_f:
       load_dict = json.load(load_f)
    cortexs = get_structure(load_dict,classname=['roi'])
    x_min, y_min, x_max, y_max = cortexs[0].bounds
    slide = openslide.OpenSlide(wsi_file)
    pas = cv2.cvtColor(
        np.asarray(slide.read_region((int(x_min), int(y_min)), 0, (1000,1000)))[:, :,
        0:3], cv2.COLOR_RGB2BGR)
    cv2.imwrite(f'/Users/fanfan/Desktop/tubule_paper_KI/KI_resubmission/bar_figure/json/{wsiid}_{x_min}_{y_min}.png',pas)
    

Extracting ['roi']: 100%|██████████████████████| 4/4 [00:00<00:00, 48629.61it/s]


In [25]:
json_files = glob.glob("/Users/fanfan/Desktop/tubule_paper_KI/KI_resubmission/bar_figure/json/16*.json")
for json_file in json_files:
    wsiid = os.path.basename(json_file).split('.json')[0]
    wsi_file = glob.glob(f"/Volumes/data/UMich/WSI/pas/{wsiid}*")[0]
    with open(json_file, 'r') as load_f:
       load_dict = json.load(load_f)
    cortexs = get_structure(load_dict,classname=['roi'])
    x_min, y_min, x_max, y_max = cortexs[0].bounds
    slide = openslide.OpenSlide(wsi_file)
    pas = cv2.cvtColor(
        np.asarray(slide.read_region((int(x_min), int(y_min)), 0, (1000,1000)))[:, :,
        0:3], cv2.COLOR_RGB2BGR)
    cv2.imwrite(f'/Users/fanfan/Desktop/tubule_paper_KI/KI_resubmission/bar_figure/json/{wsiid}_{x_min}_{y_min}.png',pas)
    

Extracting ['roi']: 100%|███████████████████████| 2/2 [00:00<00:00, 7810.62it/s]


In [ ]:
with open(json_file, 'r') as load_f:
       load_dict = json.load(load_f)
    wsiid = (os.path.basename(json_file)).split('.json')[0]
    bid = wsiid
    # for i in range(len(column)):
    #     info = column[i]
    #     filename = info['filename']
    #     if (wsiid in filename):
    #         bid = info['biopsyid']
    print(f'###### {bid} ##### {wsiid} #######')
    if (os.path.exists(f'{par_path}/{bid}')==True):
        print('FILES EXIST!')
    else:
        slide = read_wsi_curegn(wsiid,wsi_path_par)
        create_all_folders(bid,par_path)
        print('Get cortex data...')